In [1]:
import ast
import datetime as dt
import os
import sys

import pandas as pd
from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
sys.path.insert(0, '..')
import gps

In [2]:
files = [os.path.join('gmaps_cache', f) for f in os.listdir('gmaps_cache')]

dfs = []
for f in files:
    a = pd.read_csv(f)

    a['dtRetrieved'] = dt.datetime.now()
    a['radius']  = 50
    a['rankby']  = 'prominence'
    a['source']  = 'Google Places'

    dfs.append(a)
    
df = pd.concat(dfs, sort=True)
df['mask_'] = [not x for x in df.loc[:, ['lat', 'lon']].duplicated(keep='first')]
df.head()

In [3]:
records = [
    gps.PlaceRequest().from_tuple(t, gps.ApiSource.GMAPS) 
    for t in df.loc[df.mask_].itertuples()
]

for r in tqdm(records):
    c = ast.literal_eval(r.content)
    c = json.dumps(c)
    r.content = c

In [4]:
for r in tqdm(records):
    with gps.session_scope() as session:
        session.add(r)

### example recall the entire cache

In [5]:
with gps.session_scope() as s:
    t = pd.concat([ti.dataframe for ti in s.query(gps.PlaceRequest).all()], sort=True)

cats = pd.DataFrame([gps.parse_gmap_types(c) for c in t.content])
cats.loc[cats.rank_order != -1].head(10)